In [2]:
#imports
import websockets
import asyncio
import time
import json
import re
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
load_dotenv()
import os

In [3]:
#alpaca url 
base_url = 'https://paper-api.alpaca.markets'
#alpaca api keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
#alpaca object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url
)

In [ ]:
#Function to gather live feed and trade CPI
async def listen():
    #url we gather cpi data from
    url = 'wss://www.madnews.io/ws'

    #async continually pings the websocket asking to receive new messages 
    async with websockets.connect(url) as websocket:
        while True:
            msg = await websocket.recv()
            data = json.loads(msg)
            #regular expression used to clean CPI numbers
            matches = re.match(r"U\.S\.\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+?\nU\.S\.\s+CORE\s+CPI:\s+\+((\d+)(?:\.\d+)?)%.+", data['title'])
            t0 = time.time()

            cpi = int(float(matches.group(1)))
            core_cpi = int(float(matches.group(3)))

            #conditionals to determine short/long
            if cpi >= 6.5:
                alpaca_api.submit_order(
                symbol='SPY',
                qty=100,  
                side='sell',
                type='limit',
                limit_price= 375,
                time_in_force='day',
                extended_hours=True,
            )
                t1 = time.time()
                print(t1-t0)
                break 
            elif cpi <= 6.4:
                alpaca_api.submit_order(
                symbol='BTCUSD',
                qty=15,  
                side='buy',
                type='market',
                time_in_force='gtc',
            )
                t1 = time.time()
                print(t1-t0)
                break
            else:
                print("No trade to be made given current data.")
                t1 = time.time()
                print(t1-t0)
                break